# Benchmark of various outlier detection model thresholders

### The models are evaluated by ROC, Precision @ n and execution time on 17 benchmark datasets. All datasets are split (60% for training and 40% for testing). 

The thresholders covered in this example include:

1. **IQR: Inter-Quartile Region** 
2. **MAD: Median Absolute Deviation**
3. **FWFM: Full Width at Full Minimum**
4. **YJ: Yeo-Johnson Transformation**
5. **ZSCORE: Z Score**
6. **AUCP: Area Under the Curve Percentage**
7. **QMCD: Quasi-Monte Carlo Discreperancy**
8. **FGD: Fixed Gradient Descent**
9. **DSN: Distance Shift from Normal**
10. **CLF: Trained Classifier**
11. **FILTER: Filtering Based**
12. **WIND: Topological Winding Number**
13. **EB: Elliptical Boundary**
14. **REGR: Regression Intercept**
15. **BOOT: Bootstrap Method**
16. **MCST: Monte Carlo Statistical Tests**
17. **HIST: Histogram Based Methods**
18. **MOLL: Mollifier**
19. **CHAU: Chauvenet's Criterion**
20. **GESD: Generalized Extreme Studentized Deviate**
21. **MTT: Modified Thompson Tau Test**
22. **KARCH: Karcher Mean**
23. **OCSVM: One-Class SVM**
24. **CLUST: Clustering**
25. **DECOMP: Decomposition**
26. **META: Meta-model**

In [2]:
from __future__ import division
from __future__ import print_function

import os
import sys
from time import time

# temporary solution for relative imports in case pyod is not installed
# if pythresh is installed, no need to use the following line
sys.path.append(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), '..')))

import numpy as np
from numpy import percentile
import matplotlib.pyplot as plt
import matplotlib.font_manager
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

from pyod.models.knn import KNN
from pyod.models.iforest import IForest

from pythresh.thresholds.iqr import IQR
from pythresh.thresholds.mad import MAD
from pythresh.thresholds.fwfm import FWFM
from pythresh.thresholds.yj import YJ
from pythresh.thresholds.zscore import ZSCORE
from pythresh.thresholds.aucp import AUCP
from pythresh.thresholds.qmcd import QMCD
from pythresh.thresholds.fgd import FGD
from pythresh.thresholds.dsn import DSN
from pythresh.thresholds.clf import CLF
from pythresh.thresholds.filter import FILTER
from pythresh.thresholds.wind import WIND
from pythresh.thresholds.eb import EB
from pythresh.thresholds.regr import REGR
from pythresh.thresholds.boot import BOOT
from pythresh.thresholds.mcst import MCST
from pythresh.thresholds.hist import HIST
from pythresh.thresholds.moll import MOLL
from pythresh.thresholds.chau import CHAU
from pythresh.thresholds.gesd import GESD
from pythresh.thresholds.mtt import MTT
from pythresh.thresholds.karch import KARCH
from pythresh.thresholds.ocsvm import OCSVM
from pythresh.thresholds.clust import CLUST
from pythresh.thresholds.decomp import DECOMP
from pythresh.thresholds.meta import META

from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

INFO: Using numpy backend


In [3]:
# Define data file and read X and y
mat_file_list = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat',
                 'ionosphere.mat',
                 'letter.mat',
                 'lympho.mat',
                 'mnist.mat',
                 'musk.mat',
                 'optdigits.mat',
                 'pendigits.mat',
                 'pima.mat',
                 'satellite.mat',
                 'satimage-2.mat',
                 'vertebral.mat',
                 'vowels.mat',
                 'wbc.mat']

# Define nine outlier detection tools to be compared
random_state = np.random.RandomState(42)

df_columns = ['Data', '#Samples', '# Dimensions', 'Outlier Perc', 'IQR', 'MAD', 'FWFM', 
              'YJ', 'ZSCORE', 'AUCP', 'QMCD', 'FGD', 'DSN', 'CLF', 'FILTER', 'WIND', 
              'EB', 'REGR', 'BOOT', 'MCST', 'HIST', 'MOLL', 'CHAU', 'GESD', 'MTT', 
              'KARCH', 'OCSVM', 'CLUST', 'DECOMP', 'META']

roc_df = pd.DataFrame(columns=df_columns)
prn_df = pd.DataFrame(columns=df_columns)
time_df = pd.DataFrame(columns=df_columns)

clf = IForest()


for mat_file in mat_file_list:
    print("\n... Processing", mat_file, '...')
    mat = loadmat(os.path.join('data', mat_file))

    X = mat['X']
    y = mat['y'].ravel()
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)

    # construct containers for saving results
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]

    # 60% data for training and 40% for testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,
                                                        random_state=random_state)

    # standardizing data for processing
    X_train_norm, X_test_norm = standardizer(X_train, X_test)

    thresholders = {
        'Inter-Quartile Region (IQR)':IQR(),
        'Median Absolute Deviation (MAD)':MAD(),
        'Full Width at Full Minimum (FWFM)':FWFM(),
        'Yeo-Johnson Transformation (YJ)': YJ(),
        'Z Score (ZSCORE)': ZSCORE(),
        'AUC Percentage (AUCP)': AUCP(),
        'Quasi-Monte Carlo Discreperancy (QMCD)': QMCD(),
        'Fixed Gradient Descent (FGD)': FGD(),
        'Distance Shift from Normal (DSN)': DSN(),
        'Trained Classifier (CLF)': CLF(),
        'Filtering Based (FILTER)': FILTER(),
        'Topological Winding Number (WIND)': WIND(),
        'Elliptical Boundary (EB)': EB(),
        'Regression Intercept (REGR)': REGR(),
        'Bootstrap Method (BOOT)': BOOT(),
        'Monte Carlo Statistical Tests (MCST)': MCST(),
        'Histogram Based Methods (HIST)': HIST(),
        'Mollifier (MOLL)': MOLL(),
        "Chauvenet's Criterion (CHAU)": CHAU(),
        'Generalized Extreme Studentized Deviate (GESD)': GESD(),
        'Modified Thompson Tau Test (MTT)': MTT(),
        'Karcher Mean (KARCH)': KARCH(),
        'One-Class SVM (OCSVM)': OCSVM(),
        'Clustering (CLUST)': CLUST(),
        'Decomposition (DECOMP)': DECOMP(),
        'Meta-model (META)': META()
    }
    
    clf.fit(X_train_norm)
    scores = clf.decision_scores_
    
    for thres_name, thres in thresholders.items():
        t0 = time()
        pred = thres.eval(scores)
        contam = np.sum(pred)/len(pred)
        
        if contam<=0: contam=1e-3
        if contam>0.5: contam=0.5
        
        clf = IForest(contamination=contam)
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1 - t0, ndigits=4)
        time_list.append(duration)

        roc = round(roc_auc_score(y_test, test_scores), ndigits=4)
        prn = round(precision_n_scores(y_test, test_scores), ndigits=4)

        #print('{thres_name} ROC:{roc}, precision @ rank n:{prn}, '
        #      'execution time: {duration}s'.format(
        #thres_name=thres_name, roc=roc, prn=prn, duration=duration))

        roc_list.append(roc)
        prn_list.append(prn)

    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)

    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)

    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat ...

... Processing cardio.mat ...

... Processing glass.mat ...

... Processing ionosphere.mat ...

... Processing letter.mat ...

... Processing lympho.mat ...

... Processing mnist.mat ...

... Processing musk.mat ...

... Processing optdigits.mat ...

... Processing pendigits.mat ...

... Processing pima.mat ...

... Processing satellite.mat ...

... Processing satimage-2.mat ...

... Processing vertebral.mat ...

... Processing vowels.mat ...

... Processing wbc.mat ...


In [4]:
print('Time complexity')
time_df

Time complexity


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META
0,arrhythmia,452,274,14.6018,0.7916,0.792,0.8645,0.9836,0.7561,0.866,...,0.7545,0.8746,0.7979,0.9135,0.7604,0.7813,1.0321,0.8243,0.8213,3.2182
0,cardio,1831,21,9.6122,0.7902,0.7317,0.8499,1.2712,0.7204,0.8888,...,0.6974,0.705,0.7158,1.0405,0.8012,0.8321,1.268,0.9391,1.0482,3.2978
0,glass,214,9,4.2056,0.555,0.5402,0.5251,0.8689,0.5338,0.6223,...,0.5421,0.5812,0.5281,0.5754,0.5254,0.5594,0.6747,0.5238,0.5457,2.8945
0,ionosphere,351,33,35.8974,0.587,0.5958,0.5998,0.8162,0.5833,0.608,...,0.5918,0.581,0.6048,0.6555,0.6078,0.5873,0.7644,0.5849,0.61,2.867
0,letter,1600,32,6.25,0.7236,0.7242,0.8166,1.4029,0.7043,0.9465,...,0.7686,0.7435,0.8015,1.024,0.7486,0.8197,1.172,0.8584,1.079,3.4154
0,lympho,148,18,4.0541,0.6208,0.6188,0.6288,0.8317,0.7041,0.6469,...,0.5265,0.543,0.544,0.5689,0.5378,0.6638,0.7011,0.5276,0.6196,3.0538
0,mnist,7603,100,9.2069,2.4641,2.399,4.6129,5.7485,2.3419,4.6573,...,2.3326,2.5465,2.4583,3.8665,2.3965,3.3263,5.6389,7.1912,6.2451,5.5736
0,musk,3062,166,3.1679,1.6556,1.6935,1.9703,2.7414,1.7626,2.26,...,1.7397,1.7862,1.8548,2.3014,1.7503,2.0552,3.0953,2.2176,2.3006,4.1617
0,optdigits,5216,64,2.8758,1.6228,1.5946,2.651,3.9215,1.5673,2.7288,...,1.5722,1.5989,1.5822,2.5781,1.7175,2.1189,3.2735,3.8383,3.3431,4.3426
0,pendigits,6870,16,2.2707,1.4267,1.1775,2.8684,3.9494,1.2167,2.9454,...,1.1704,1.2082,1.2332,2.4362,1.2647,2.0245,4.1419,6.3293,4.1291,4.0161


In [5]:
print('ROC Performance')
roc_df

ROC Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META
0,arrhythmia,452,274,14.6018,0.8392,0.8648,0.8466,0.8588,0.852,0.8417,...,0.8492,0.8375,0.8548,0.8576,0.8147,0.8403,0.8172,0.8466,0.8345,0.8597
0,cardio,1831,21,9.6122,0.9294,0.9102,0.9381,0.9161,0.9181,0.9082,...,0.9236,0.9038,0.942,0.9304,0.9261,0.9034,0.9179,0.9274,0.901,0.9221
0,glass,214,9,4.2056,0.6346,0.6765,0.6222,0.6667,0.6617,0.6889,...,0.642,0.6469,0.7037,0.6494,0.6346,0.6519,0.642,0.6296,0.6198,0.6395
0,ionosphere,351,33,35.8974,0.7842,0.7913,0.8018,0.7936,0.7796,0.787,...,0.7684,0.7757,0.768,0.7817,0.768,0.7799,0.7757,0.7888,0.7723,0.7886
0,letter,1600,32,6.25,0.643,0.6364,0.5975,0.6335,0.5952,0.6258,...,0.5988,0.5677,0.6585,0.5961,0.6206,0.594,0.6753,0.627,0.6129,0.6883
0,lympho,148,18,4.0541,0.9942,1.0,0.9942,1.0,1.0,1.0,...,1.0,1.0,0.9942,1.0,1.0,0.9942,0.9942,0.9942,1.0,1.0
0,mnist,7603,100,9.2069,0.771,0.7885,0.7896,0.8006,0.825,0.7748,...,0.8053,0.8196,0.7826,0.8127,0.7941,0.7692,0.7976,0.8062,0.7882,0.8005
0,musk,3062,166,3.1679,1.0,0.9997,1.0,0.9992,1.0,0.9999,...,0.9886,0.9996,0.9997,0.9997,0.9988,1.0,1.0,1.0,1.0,1.0
0,optdigits,5216,64,2.8758,0.7461,0.6624,0.6121,0.7007,0.6863,0.7207,...,0.7732,0.6948,0.7126,0.7197,0.7483,0.6892,0.7659,0.7221,0.7606,0.6923
0,pendigits,6870,16,2.2707,0.9595,0.9487,0.9437,0.9331,0.9667,0.9459,...,0.9497,0.9349,0.9256,0.9442,0.9437,0.959,0.9222,0.9528,0.943,0.9452


In [6]:
print('Precision @ n Performance')
prn_df

Precision @ n Performance


,Data,#Samples,# Dimensions,Outlier Perc,IQR,MAD,FWFM,YJ,ZSCORE,AUCP,...,HIST,MOLL,CHAU,GESD,MTT,KARCH,OCSVM,CLUST,DECOMP,META
0,arrhythmia,452,274,14.6018,0.5357,0.5357,0.5714,0.6071,0.5357,0.5357,...,0.5714,0.5714,0.6071,0.5357,0.5357,0.5357,0.5,0.5714,0.5357,0.5357
0,cardio,1831,21,9.6122,0.5286,0.4143,0.5571,0.4714,0.4571,0.4857,...,0.5,0.4714,0.5857,0.5,0.5286,0.4429,0.4857,0.4857,0.4,0.5143
0,glass,214,9,4.2056,0.2,0.2,0.2,0.2,0.2,0.2,...,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2
0,ionosphere,351,33,35.8974,0.5435,0.5652,0.587,0.5652,0.5435,0.5652,...,0.5435,0.5217,0.5217,0.587,0.5,0.5217,0.5217,0.5435,0.5435,0.587
0,letter,1600,32,6.25,0.0976,0.0732,0.0732,0.122,0.0976,0.0488,...,0.0488,0.0732,0.0976,0.0488,0.0488,0.0488,0.0976,0.122,0.0976,0.0732
0,lympho,148,18,4.0541,0.6667,1.0,0.6667,1.0,1.0,1.0,...,1.0,1.0,0.6667,1.0,1.0,0.6667,0.6667,0.6667,1.0,1.0
0,mnist,7603,100,9.2069,0.2259,0.2815,0.2889,0.2778,0.3222,0.2407,...,0.3185,0.3556,0.2556,0.3185,0.2852,0.2407,0.2815,0.3111,0.2704,0.2815
0,musk,3062,166,3.1679,1.0,0.9512,1.0,0.925,1.0,0.9756,...,0.7561,0.9024,0.9756,0.9512,0.9024,1.0,1.0,1.0,1.0,0.975
0,optdigits,5216,64,2.8758,0.0154,0.0154,0.0,0.0308,0.0154,0.0308,...,0.0154,0.0154,0.0154,0.0462,0.0154,0.0154,0.0308,0.0154,0.0462,0.0154
0,pendigits,6870,16,2.2707,0.3387,0.2903,0.2903,0.2419,0.3065,0.2419,...,0.2742,0.2581,0.2097,0.2581,0.2419,0.3387,0.2419,0.2258,0.2742,0.3226
